# QA Quick Start

In this QA Quick Start notebook, we take a first look at semantic ingestion, RAG, and retrieval, presenting basic usage patterns.

For more advanced usage, check out [QA Deep Dive](./qa_deep_dive.ipynb).

### Access required

The content of this notebook requires access to Deep Search capabilities which are not
available on the public access system.

[Contact us](https://ds4sd.github.io) if you are interested in exploring
these Deep Search capabilities.


### GenAI Integration required

When interacting with the virtual assistant, Deep Search requires a connection to a Generative AI API. Currently, we support connections to [watsonx.ai](https://www.ibm.com/products/watsonx-ai) or the IBM-internal GenAI platform BAM.

Deep Search allows custom GenAI configurations for each project.
In the following example you will require to work in a project which has such GenAI capabilities activated.

### Set notebook parameters


In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

# profile to use (user's active profile will be used by default):
PROFILE_NAME = os.environ.get("DS_NB_PROFILE")

PROJ_KEY = os.environ["DS_NB_PROJ_KEY"]  # project to use
INDEX_KEY = os.environ["DS_NB_SEM_ON_IDX_KEY"]

# set the doc hash when targeting a specific document
DOC_HASH = os.environ.get("DS_NB_SEM_ON_IDX_DOC_HASH")
QUESTION = os.environ["DS_NB_SEM_ON_QUESTION"]

from pydantic import TypeAdapter

# whether to skip any already semantically ingested docs:
SKIP_INGESTED_DOCS = TypeAdapter(bool).validate_python(
    os.environ.get("DS_NB_SKIP_INGESTED_DOCS", True)
)
RETR_K = os.environ.get("DS_NB_RETR_K", 3)  # number of search results to retrieve
GEN_TIMEOUT = os.environ.get("DB_NB_GEN_TIMEOUT")  # generation timeout in secs

### Import example dependencies

In [9]:
import rich
from deepsearch.cps.client.api import CpsApi
from deepsearch.cps.client.components.documents import create_private_data_source
from deepsearch.cps.queries import RAGQuery, SemanticQuery
from deepsearch.cps.queries.results import RAGResult, SearchResult

### Connect to Deep Search

In [10]:
api = CpsApi.from_env(profile_name=PROFILE_NAME)

### Prepare data source

The cell below shows how to configure a private data source, i.e. either a whole private collection (in which case `document_hash` should be `None` or omitted) or a given doc within one.

For more details on data sources check out [QA Deep Dive](./qa_deep_dive.ipynb).

In [11]:
data_source = create_private_data_source(
    proj_key=PROJ_KEY,
    index_key=INDEX_KEY,
    document_hash=DOC_HASH,
)

### Ingestion

If your data source has not yet been semantically indexed, you can ingest it into the vector DB as shown below. Otherwise you can skip this step.

Ingestion of already indexed docs is controlled via param `skip_ingested_docs`. 

Particularly when indexing whole collections, note that the larger the data source, the longer the ingestion duration.

In [12]:
task = api.documents.semantic_ingest(
    project=PROJ_KEY,
    data_source=data_source,
    skip_ingested_docs=SKIP_INGESTED_DOCS,
)

# wait for the ingestion task to finish
api.tasks.wait_for(PROJ_KEY, task.task_id)

{'ing_out': {}}

### RAG

The cell below demonstrates basic RAG usage.

For more advanced usage and parametrization, check out [QA Deep Dive](./qa_deep_dive.ipynb).

In [13]:
query = RAGQuery(
    question=QUESTION,
    project=PROJ_KEY,
    data_source=data_source,
    retr_k=RETR_K,  # optional
    gen_timeout=GEN_TIMEOUT,  # optional
)

api_output = api.queries.run(query)
result = RAGResult.from_api_output(api_output)

rich.print(QUESTION)
rich.print(result)

Where was the first European IBM lab located?

RAGResult(
    answers=[
        RAGAnswerItem(
            answer='The first European IBM lab was located in Adliswil, Switzerland, near Zurich.',
            grounding=RAGGroundingInfo(
                retr_items=[
                    SearchResultItem(
                        doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
                        chunk='Switzerland\nIn 1956, IBM opened their first European research laboratory in 
Adliswil, Switzerland, near Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich 
lab is staffed by a multicultural and interdisciplinary team of a few hundred permanent research staff members, 
graduate students and post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a 
Client Center (formerly the Industry Solutions Lab), an executive briefing facility demonstrating technology 
prototypes and solutions.',
                        main_path='main-text.71',
                        path_group=['main-text.69', 'main-text.71'],
                        source_is_text=True
                    )
                ],
                gen_ctx_paths=[
                    'main-text.60',
                    'main-text.61',
                    'main-text.62',
                    'main-text.63',
                    'main-text.64',
                    'main-text.65',
                    'main-text.66',
                    'main-text.67',
                    'main-text.68',
                    'main-text.69',
                    'main-text.70',
                    'main-text.71',
                    'main-text.72',
                    'main-text.73',
                    'main-text.74',
                    'main-text.75',
                    'main-text.76',
                    'main-text.77',
                    'main-text.78'
                ]
            ),
            prompt=None
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='Switzerland\nIn 1956, IBM opened their first European research laboratory in Adliswil, 
Switzerland, near Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich lab is 
staffed by a multicultural and interdisciplinary team of a few hundred permanent research staff members, graduate 
students and post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a Client Center
(formerly the Industry Solutions Lab), an executive briefing facility demonstrating technology prototypes and 
solutions.',
            main_path='main-text.71',
            path_group=['main-text.69', 'main-text.71'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='Switzerland\nIBM Research-Zurich (previously called IBM Zurich Research Laboratory, ZRL) is the 
European branch of IBM Research. It was opened in 1956 and is located in Rüschlikon, near Zurich, Switzerland.',
            main_path='main-text.70',
            path_group=['main-text.69', 'main-text.70'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk="History\nThe roots of today's IBM Research began with the 1945 opening of the Watson Scientific 
Computing Laboratory at Columbia University. $^{[4]}$ This was the first IBM laboratory devoted to pure science and
later expanded into additional IBM Research locations in Westchester County, New York, starting in the 1950s, 
$^{[5][6]}$ including the Thomas J. Watson Research Center in 1961. [5][6]",
            main_path='main-text.8',
            path_group=['main-text.7', 'main-text.8'],
            source_is_text=True
        )
    ]
)

### Semantic retrieval

In certain cases, a user may only be interested in the semantic retrieval part, instead of the whole RAG pipeline.

Basic semantic retrieval usage is shown below:

In [14]:
query = SemanticQuery(
    question=QUESTION,
    project=PROJ_KEY,
    data_source=data_source,
    retr_k=RETR_K,  # optional
)
api_output = api.queries.run(query)
result = SearchResult.from_api_output(api_output)

rich.print(QUESTION)
rich.print(result)

Where was the first European IBM lab located?

SearchResult(
    search_result_items=[
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='Switzerland\nIn 1956, IBM opened their first European research laboratory in Adliswil, 
Switzerland, near Zurich. The lab moved to its own campus in neighboring Rüschlikon in 1962. The Zurich lab is 
staffed by a multicultural and interdisciplinary team of a few hundred permanent research staff members, graduate 
students and post-doctoral fellows, representing about 45 nationalities. Collocated with the lab is a Client Center
(formerly the Industry Solutions Lab), an executive briefing facility demonstrating technology prototypes and 
solutions.',
            main_path='main-text.71',
            path_group=['main-text.69', 'main-text.71'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk='Switzerland\nIBM Research-Zurich (previously called IBM Zurich Research Laboratory, ZRL) is the 
European branch of IBM Research. It was opened in 1956 and is located in Rüschlikon, near Zurich, Switzerland.',
            main_path='main-text.70',
            path_group=['main-text.69', 'main-text.70'],
            source_is_text=True
        ),
        SearchResultItem(
            doc_hash='b30bc667a324ae111d025526563b674a8d3fd869bc07c8fd204aa95b05d41f0c',
            chunk="History\nThe roots of today's IBM Research began with the 1945 opening of the Watson Scientific 
Computing Laboratory at Columbia University. $^{[4]}$ This was the first IBM laboratory devoted to pure science and
later expanded into additional IBM Research locations in Westchester County, New York, starting in the 1950s, 
$^{[5][6]}$ including the Thomas J. Watson Research Center in 1961. [5][6]",
            main_path='main-text.8',
            path_group=['main-text.7', 'main-text.8'],
            source_is_text=True
        )
    ]
)